# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.


In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

# **31_EYE**

In [ ]:
df_31_EYE = pd.read_csv('data/STData/31/31_EYE.csv')

In [ ]:
df_31_EYE.head()

In [ ]:
df_31_EYE.shape

In [ ]:
df_31_EYE.columns

In [ ]:
df_31_EYE.info()

In [ ]:
df_31_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_31_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_31_EYE['QuestionKey'].unique()

In [ ]:
df_31_EYE['Timestamp'] = pd.to_datetime(df_31_EYE['Timestamp'])

In [ ]:
df_31_EYE.head(3)

In [ ]:
df_31_EYE['QuestionKey'] = df_31_EYE['QuestionKey'].fillna('None')

In [ ]:
df_31_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_31_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_31_EYE.isnull().sum()

In [ ]:
df_31_EYE.dropna(inplace=True)

In [ ]:
df_31_EYE.head()

In [ ]:
df_31_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_31_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_31_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_31_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_31_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_31_EYE['ET_ValidityRight'].unique()

In [ ]:
df_31_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_31_EYE['ET_ValidityLeft'].value_counts().index, y=df_31_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_31_EYE['ET_ValidityRight'].value_counts().index, y=df_31_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_31_EYE['ET_ValidityLeft'] = df_31_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_31_EYE['ET_ValidityRight'] = df_31_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_31_EYE.head(3)

In [ ]:
df_31_EYE.describe()

In [ ]:
df_31_EYE[df_31_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_31_EYE[df_31_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_31_EYE[df_31_EYE['ET_ValidityLeft'] == 1].shape[0] / df_31_EYE.shape[0]

In [ ]:
df_31_EYE[df_31_EYE['ET_ValidityRight'] == 1].shape[0] / df_31_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_31_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_31_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_31_EYE[df_31_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_31_EYE[df_31_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_31_EYE[df_31_EYE['ET_PupilLeft'] == -1].shape[0] / df_31_EYE.shape[0]

In [ ]:
df_31_EYE[df_31_EYE['ET_PupilRight'] == -1].shape[0] / df_31_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_31_EYE[df_31_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_31_EYE[df_31_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_31_EYE['ET_PupilLeft_validity'] = df_31_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_31_EYE['ET_PupilRight_validity'] = df_31_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_31_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_31_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_31_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_31_EYE['ET_PupilLeft_validity'] = df_31_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_31_EYE['ET_PupilRight_validity'] = df_31_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_31_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_31_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_31_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_31_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_31_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_31_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_31_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_31_EYE['ET_PupilLeft_validity'] = df_31_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_31_EYE['ET_PupilRight_validity'] = df_31_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_31_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_31_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_31_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_31_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_31_EYE['Timestamp'], df_31_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_31_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_31_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_31_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_31_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_31_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_31_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_31_EYE[col] = df_31_EYE[col].fillna(df_31_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_31_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_31_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_31_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_31_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_31_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_31_EYE['Timestamp'], df_31_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_31_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_31_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_31_EYE)
plt.show()

# **32_EYE**

In [ ]:
df_32_EYE = pd.read_csv('data/STData/32/32_EYE.csv')

In [ ]:
df_32_EYE.head()

In [ ]:
df_32_EYE.shape

In [ ]:
df_32_EYE.columns

In [ ]:
df_32_EYE.info()

In [ ]:
df_32_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_32_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_32_EYE['QuestionKey'].unique()

In [ ]:
df_32_EYE['Timestamp'] = pd.to_datetime(df_32_EYE['Timestamp'])

In [ ]:
df_32_EYE.head(3)

In [ ]:
df_32_EYE['QuestionKey'] = df_32_EYE['QuestionKey'].fillna('None')

In [ ]:
df_32_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_32_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_32_EYE.isnull().sum()

In [ ]:
df_32_EYE.dropna(inplace=True)

In [ ]:
df_32_EYE.head()

In [ ]:
df_32_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_32_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_32_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_32_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_32_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_32_EYE['ET_ValidityRight'].unique()

In [ ]:
df_32_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_32_EYE['ET_ValidityLeft'].value_counts().index, y=df_32_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_32_EYE['ET_ValidityRight'].value_counts().index, y=df_32_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_32_EYE['ET_ValidityLeft'] = df_32_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_32_EYE['ET_ValidityRight'] = df_32_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_32_EYE.head(3)

In [ ]:
df_32_EYE.describe()

In [ ]:
df_32_EYE[df_32_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_32_EYE[df_32_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_32_EYE[df_32_EYE['ET_ValidityLeft'] == 1].shape[0] / df_32_EYE.shape[0]

In [ ]:
df_32_EYE[df_32_EYE['ET_ValidityRight'] == 1].shape[0] / df_32_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_32_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_32_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_32_EYE[df_32_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_32_EYE[df_32_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_32_EYE[df_32_EYE['ET_PupilLeft'] == -1].shape[0] / df_32_EYE.shape[0]

In [ ]:
df_32_EYE[df_32_EYE['ET_PupilRight'] == -1].shape[0] / df_32_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_32_EYE[df_32_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_32_EYE[df_32_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_32_EYE['ET_PupilLeft_validity'] = df_32_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_32_EYE['ET_PupilRight_validity'] = df_32_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_32_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_32_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_32_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_32_EYE['ET_PupilLeft_validity'] = df_32_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_32_EYE['ET_PupilRight_validity'] = df_32_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_32_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_32_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_32_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_32_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_32_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_32_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_32_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_32_EYE['ET_PupilLeft_validity'] = df_32_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_32_EYE['ET_PupilRight_validity'] = df_32_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_32_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_32_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_32_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_32_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_32_EYE['Timestamp'], df_32_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_32_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_32_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_32_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_32_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_32_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_32_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_32_EYE[col] = df_32_EYE[col].fillna(df_32_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_32_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_32_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_32_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_32_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_32_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_32_EYE['Timestamp'], df_32_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_32_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_32_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_32_EYE)
plt.show()

# **33_EYE**

In [ ]:
df_33_EYE = pd.read_csv('data/STData/33/33_EYE.csv')

In [ ]:
df_33_EYE.head()

In [ ]:
df_33_EYE.shape

In [ ]:
df_33_EYE.columns

In [ ]:
df_33_EYE.info()

In [ ]:
df_33_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_33_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_33_EYE['QuestionKey'].unique()

In [ ]:
df_33_EYE['Timestamp'] = pd.to_datetime(df_33_EYE['Timestamp'])

In [ ]:
df_33_EYE.head(3)

In [ ]:
df_33_EYE['QuestionKey'] = df_33_EYE['QuestionKey'].fillna('None')

In [ ]:
df_33_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_33_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_33_EYE.isnull().sum()

In [ ]:
df_33_EYE.dropna(inplace=True)

In [ ]:
df_33_EYE.head()

In [ ]:
df_33_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_33_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_33_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_33_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_33_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_33_EYE['ET_ValidityRight'].unique()

In [ ]:
df_33_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_33_EYE['ET_ValidityLeft'].value_counts().index, y=df_33_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_33_EYE['ET_ValidityRight'].value_counts().index, y=df_33_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_33_EYE['ET_ValidityLeft'] = df_33_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_33_EYE['ET_ValidityRight'] = df_33_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_33_EYE.head(3)

In [ ]:
df_33_EYE.describe()

In [ ]:
df_33_EYE[df_33_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_33_EYE[df_33_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_33_EYE[df_33_EYE['ET_ValidityLeft'] == 1].shape[0] / df_33_EYE.shape[0]

In [ ]:
df_33_EYE[df_33_EYE['ET_ValidityRight'] == 1].shape[0] / df_33_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_33_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_33_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_33_EYE[df_33_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_33_EYE[df_33_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_33_EYE[df_33_EYE['ET_PupilLeft'] == -1].shape[0] / df_33_EYE.shape[0]

In [ ]:
df_33_EYE[df_33_EYE['ET_PupilRight'] == -1].shape[0] / df_33_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_33_EYE[df_33_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_33_EYE[df_33_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_33_EYE['ET_PupilLeft_validity'] = df_33_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_33_EYE['ET_PupilRight_validity'] = df_33_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_33_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_33_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_33_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_33_EYE['ET_PupilLeft_validity'] = df_33_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_33_EYE['ET_PupilRight_validity'] = df_33_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_33_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_33_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_33_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_33_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_33_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_33_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_33_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_33_EYE['ET_PupilLeft_validity'] = df_33_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_33_EYE['ET_PupilRight_validity'] = df_33_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_33_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_33_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_33_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_33_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_33_EYE['Timestamp'], df_33_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_33_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_33_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_33_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_33_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_33_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_33_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_33_EYE[col] = df_33_EYE[col].fillna(df_33_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_33_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_33_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_33_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_33_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_33_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_33_EYE['Timestamp'], df_33_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_33_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_33_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_33_EYE)
plt.show()

# **34_EYE**

In [ ]:
df_34_EYE = pd.read_csv('data/STData/34/34_EYE.csv')

In [ ]:
df_34_EYE.head()

In [ ]:
df_34_EYE.shape

In [ ]:
df_34_EYE.columns

In [ ]:
df_34_EYE.info()

In [ ]:
df_34_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_34_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_34_EYE['QuestionKey'].unique()

In [ ]:
df_34_EYE['Timestamp'] = pd.to_datetime(df_34_EYE['Timestamp'])

In [ ]:
df_34_EYE.head(3)

In [ ]:
df_34_EYE['QuestionKey'] = df_34_EYE['QuestionKey'].fillna('None')

In [ ]:
df_34_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_34_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_34_EYE.isnull().sum()

In [ ]:
df_34_EYE.dropna(inplace=True)

In [ ]:
df_34_EYE.head()

In [ ]:
df_34_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_34_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_34_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_34_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_34_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_34_EYE['ET_ValidityRight'].unique()

In [ ]:
df_34_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_34_EYE['ET_ValidityLeft'].value_counts().index, y=df_34_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_34_EYE['ET_ValidityRight'].value_counts().index, y=df_34_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_34_EYE['ET_ValidityLeft'] = df_34_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_34_EYE['ET_ValidityRight'] = df_34_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_34_EYE.head(3)

In [ ]:
df_34_EYE.describe()

In [ ]:
df_34_EYE[df_34_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_34_EYE[df_34_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_34_EYE[df_34_EYE['ET_ValidityLeft'] == 1].shape[0] / df_34_EYE.shape[0]

In [ ]:
df_34_EYE[df_34_EYE['ET_ValidityRight'] == 1].shape[0] / df_34_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_34_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_34_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_34_EYE[df_34_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_34_EYE[df_34_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_34_EYE[df_34_EYE['ET_PupilLeft'] == -1].shape[0] / df_34_EYE.shape[0]

In [ ]:
df_34_EYE[df_34_EYE['ET_PupilRight'] == -1].shape[0] / df_34_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_34_EYE[df_34_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_34_EYE[df_34_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_34_EYE['ET_PupilLeft_validity'] = df_34_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_34_EYE['ET_PupilRight_validity'] = df_34_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_34_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_34_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_34_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_34_EYE['ET_PupilLeft_validity'] = df_34_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_34_EYE['ET_PupilRight_validity'] = df_34_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_34_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_34_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_34_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_34_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_34_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_34_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_34_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_34_EYE['ET_PupilLeft_validity'] = df_34_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_34_EYE['ET_PupilRight_validity'] = df_34_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_34_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_34_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_34_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_34_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_34_EYE['Timestamp'], df_34_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_34_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_34_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_34_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_34_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_34_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_34_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_34_EYE[col] = df_34_EYE[col].fillna(df_34_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_34_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_34_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_34_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_34_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_34_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_34_EYE['Timestamp'], df_34_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_34_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_34_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_34_EYE)
plt.show()

# **35_EYE**

In [ ]:
df_35_EYE = pd.read_csv('data/STData/35/35_EYE.csv')

In [ ]:
df_35_EYE.head()

In [ ]:
df_35_EYE.shape

In [ ]:
df_35_EYE.columns

In [ ]:
df_35_EYE.info()

In [ ]:
df_35_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_35_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_35_EYE['QuestionKey'].unique()

In [ ]:
df_35_EYE['Timestamp'] = pd.to_datetime(df_35_EYE['Timestamp'])

In [ ]:
df_35_EYE.head(3)

In [ ]:
df_35_EYE['QuestionKey'] = df_35_EYE['QuestionKey'].fillna('None')

In [ ]:
df_35_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_35_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_35_EYE.isnull().sum()

In [ ]:
df_35_EYE.dropna(inplace=True)

In [ ]:
df_35_EYE.head()

In [ ]:
df_35_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_35_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_35_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_35_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_35_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_35_EYE['ET_ValidityRight'].unique()

In [ ]:
df_35_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_35_EYE['ET_ValidityLeft'].value_counts().index, y=df_35_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_35_EYE['ET_ValidityRight'].value_counts().index, y=df_35_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_35_EYE['ET_ValidityLeft'] = df_35_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_35_EYE['ET_ValidityRight'] = df_35_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_35_EYE.head(3)

In [ ]:
df_35_EYE.describe()

In [ ]:
df_35_EYE[df_35_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_35_EYE[df_35_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_35_EYE[df_35_EYE['ET_ValidityLeft'] == 1].shape[0] / df_35_EYE.shape[0]

In [ ]:
df_35_EYE[df_35_EYE['ET_ValidityRight'] == 1].shape[0] / df_35_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_35_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_35_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_35_EYE[df_35_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_35_EYE[df_35_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_35_EYE[df_35_EYE['ET_PupilLeft'] == -1].shape[0] / df_35_EYE.shape[0]

In [ ]:
df_35_EYE[df_35_EYE['ET_PupilRight'] == -1].shape[0] / df_35_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_35_EYE[df_35_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_35_EYE[df_35_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_35_EYE['ET_PupilLeft_validity'] = df_35_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_35_EYE['ET_PupilRight_validity'] = df_35_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_35_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_35_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_35_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_35_EYE['ET_PupilLeft_validity'] = df_35_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_35_EYE['ET_PupilRight_validity'] = df_35_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_35_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_35_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_35_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_35_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_35_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_35_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_35_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_35_EYE['ET_PupilLeft_validity'] = df_35_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_35_EYE['ET_PupilRight_validity'] = df_35_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_35_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_35_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_35_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_35_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_35_EYE['Timestamp'], df_35_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_35_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_35_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_35_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_35_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_35_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_35_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_35_EYE[col] = df_35_EYE[col].fillna(df_35_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_35_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_35_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_35_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_35_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_35_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_35_EYE['Timestamp'], df_35_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_35_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_35_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_35_EYE)
plt.show()

# **36_EYE**

In [ ]:
df_36_EYE = pd.read_csv('data/STData/36/36_EYE.csv')

In [ ]:
df_36_EYE.head()

In [ ]:
df_36_EYE.shape

In [ ]:
df_36_EYE.columns

In [ ]:
df_36_EYE.info()

In [ ]:
df_36_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_36_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_36_EYE['QuestionKey'].unique()

In [ ]:
df_36_EYE['Timestamp'] = pd.to_datetime(df_36_EYE['Timestamp'])

In [ ]:
df_36_EYE.head(3)

In [ ]:
df_36_EYE['QuestionKey'] = df_36_EYE['QuestionKey'].fillna('None')

In [ ]:
df_36_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_36_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_36_EYE.isnull().sum()

In [ ]:
df_36_EYE.dropna(inplace=True)

In [ ]:
df_36_EYE.head()

In [ ]:
df_36_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_36_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_36_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_36_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_36_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_36_EYE['ET_ValidityRight'].unique()

In [ ]:
df_36_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_36_EYE['ET_ValidityLeft'].value_counts().index, y=df_36_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_36_EYE['ET_ValidityRight'].value_counts().index, y=df_36_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_36_EYE['ET_ValidityLeft'] = df_36_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_36_EYE['ET_ValidityRight'] = df_36_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_36_EYE.head(3)

In [ ]:
df_36_EYE.describe()

In [ ]:
df_36_EYE[df_36_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_36_EYE[df_36_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_36_EYE[df_36_EYE['ET_ValidityLeft'] == 1].shape[0] / df_36_EYE.shape[0]

In [ ]:
df_36_EYE[df_36_EYE['ET_ValidityRight'] == 1].shape[0] / df_36_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_36_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_36_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_36_EYE[df_36_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_36_EYE[df_36_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_36_EYE[df_36_EYE['ET_PupilLeft'] == -1].shape[0] / df_36_EYE.shape[0]

In [ ]:
df_36_EYE[df_36_EYE['ET_PupilRight'] == -1].shape[0] / df_36_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_36_EYE[df_36_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_36_EYE[df_36_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_36_EYE['ET_PupilLeft_validity'] = df_36_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_36_EYE['ET_PupilRight_validity'] = df_36_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_36_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_36_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_36_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_36_EYE['ET_PupilLeft_validity'] = df_36_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_36_EYE['ET_PupilRight_validity'] = df_36_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_36_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_36_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_36_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_36_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_36_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_36_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_36_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_36_EYE['ET_PupilLeft_validity'] = df_36_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_36_EYE['ET_PupilRight_validity'] = df_36_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_36_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_36_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_36_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_36_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_36_EYE['Timestamp'], df_36_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_36_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_36_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_36_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_36_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_36_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_36_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_36_EYE[col] = df_36_EYE[col].fillna(df_36_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_36_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_36_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_36_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_36_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_36_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_36_EYE['Timestamp'], df_36_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_36_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_36_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_36_EYE)
plt.show()

# **37_EYE**

In [ ]:
df_37_EYE = pd.read_csv('data/STData/37/37_EYE.csv')

In [ ]:
df_37_EYE.head()

In [ ]:
df_37_EYE.shape

In [ ]:
df_37_EYE.columns

In [ ]:
df_37_EYE.info()

In [ ]:
df_37_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_37_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_37_EYE['QuestionKey'].unique()

In [ ]:
df_37_EYE['Timestamp'] = pd.to_datetime(df_37_EYE['Timestamp'])

In [ ]:
df_37_EYE.head(3)

In [ ]:
df_37_EYE['QuestionKey'] = df_37_EYE['QuestionKey'].fillna('None')

In [ ]:
df_37_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_37_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_37_EYE.isnull().sum()

In [ ]:
df_37_EYE.dropna(inplace=True)

In [ ]:
df_37_EYE.head()

In [ ]:
df_37_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_37_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_37_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_37_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_37_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_37_EYE['ET_ValidityRight'].unique()

In [ ]:
df_37_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_37_EYE['ET_ValidityLeft'].value_counts().index, y=df_37_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_37_EYE['ET_ValidityRight'].value_counts().index, y=df_37_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_37_EYE['ET_ValidityLeft'] = df_37_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_37_EYE['ET_ValidityRight'] = df_37_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_37_EYE.head(3)

In [ ]:
df_37_EYE.describe()

In [ ]:
df_37_EYE[df_37_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_37_EYE[df_37_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_37_EYE[df_37_EYE['ET_ValidityLeft'] == 1].shape[0] / df_37_EYE.shape[0]

In [ ]:
df_37_EYE[df_37_EYE['ET_ValidityRight'] == 1].shape[0] / df_37_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_37_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_37_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_37_EYE[df_37_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_37_EYE[df_37_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_37_EYE[df_37_EYE['ET_PupilLeft'] == -1].shape[0] / df_37_EYE.shape[0]

In [ ]:
df_37_EYE[df_37_EYE['ET_PupilRight'] == -1].shape[0] / df_37_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_37_EYE[df_37_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_37_EYE[df_37_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_37_EYE['ET_PupilLeft_validity'] = df_37_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_37_EYE['ET_PupilRight_validity'] = df_37_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_37_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_37_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_37_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_37_EYE['ET_PupilLeft_validity'] = df_37_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_37_EYE['ET_PupilRight_validity'] = df_37_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_37_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_37_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_37_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_37_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_37_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_37_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_37_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_37_EYE['ET_PupilLeft_validity'] = df_37_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_37_EYE['ET_PupilRight_validity'] = df_37_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_37_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_37_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_37_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_37_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_37_EYE['Timestamp'], df_37_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_37_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_37_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_37_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_37_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_37_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_37_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_37_EYE[col] = df_37_EYE[col].fillna(df_37_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_37_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_37_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_37_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_37_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_37_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_37_EYE['Timestamp'], df_37_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_37_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_37_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_37_EYE)
plt.show()

# **38_EYE**

In [ ]:
df_38_EYE = pd.read_csv('data/STData/38/38_EYE.csv')

In [ ]:
df_38_EYE.head()

In [ ]:
df_38_EYE.shape

In [ ]:
df_38_EYE.columns

In [ ]:
df_38_EYE.info()

In [ ]:
df_38_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_38_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_38_EYE['QuestionKey'].unique()

In [ ]:
df_38_EYE['Timestamp'] = pd.to_datetime(df_38_EYE['Timestamp'])

In [ ]:
df_38_EYE.head(3)

In [ ]:
df_38_EYE['QuestionKey'] = df_38_EYE['QuestionKey'].fillna('None')

In [ ]:
df_38_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_38_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_38_EYE.isnull().sum()

In [ ]:
df_38_EYE.dropna(inplace=True)

In [ ]:
df_38_EYE.head()

In [ ]:
df_38_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_38_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_38_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_38_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_38_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_38_EYE['ET_ValidityRight'].unique()

In [ ]:
df_38_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_38_EYE['ET_ValidityLeft'].value_counts().index, y=df_38_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_38_EYE['ET_ValidityRight'].value_counts().index, y=df_38_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_38_EYE['ET_ValidityLeft'] = df_38_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_38_EYE['ET_ValidityRight'] = df_38_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_38_EYE.head(3)

In [ ]:
df_38_EYE.describe()

In [ ]:
df_38_EYE[df_38_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_38_EYE[df_38_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
df_38_EYE[df_38_EYE['ET_ValidityLeft'] == 1].shape[0] / df_38_EYE.shape[0]

In [ ]:
df_38_EYE[df_38_EYE['ET_ValidityRight'] == 1].shape[0] / df_38_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_38_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_38_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_38_EYE[df_38_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_38_EYE[df_38_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_38_EYE[df_38_EYE['ET_PupilLeft'] == -1].shape[0] / df_38_EYE.shape[0]

In [ ]:
df_38_EYE[df_38_EYE['ET_PupilRight'] == -1].shape[0] / df_38_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_38_EYE[df_38_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_38_EYE[df_38_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), so instead of dropping them we can create new feature for both the `ET_PupilLeft` and `ET_PupilRight` to represent which row consist invalid `ET_PupilLeft` and `ET_PupilRight` data

In [ ]:
pupil_validity = {-1: 1 }

In [ ]:
df_38_EYE['ET_PupilLeft_validity'] = df_38_EYE['ET_PupilLeft'].map(pupil_validity)

In [ ]:
df_38_EYE['ET_PupilRight_validity'] = df_38_EYE['ET_PupilRight'].map(pupil_validity)

In [ ]:
df_38_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
df_38_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull().sum()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_38_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_38_EYE['ET_PupilLeft_validity'] = df_38_EYE['ET_PupilLeft_validity'].fillna(0)

In [ ]:
df_38_EYE['ET_PupilRight_validity'] = df_38_EYE['ET_PupilRight_validity'].fillna(0)

In [ ]:
df_38_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].head()

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(df_38_EYE[['ET_PupilLeft_validity', 'ET_PupilRight_validity']].isnull(), cmap='viridis')
plt.show()

In [ ]:
df_38_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_38_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_38_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_38_EYE['ET_ValidityLeft'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio = 1 - df_38_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_right_ratio

In [ ]:
df_38_EYE['ET_PupilLeft_validity'] = df_38_EYE['ET_PupilLeft_validity'].astype(np.int8)
df_38_EYE['ET_PupilRight_validity'] = df_38_EYE['ET_PupilRight_validity'].astype(np.int8)

# Feature Engineering and Observations

Based on the analysis of the data, we've created two new features, `ET_PupilLeft_validity` and `ET_PupilRight_validity`. These features indicate the validity of the pupil data for the left and right eyes, respectively, with a value of 1 representing invalid data (originally -1) and 0 representing valid data.

The heatmaps above visually demonstrate the distribution of -1 and 1 values across the dataset. We observed that:
- The `-1` values are concentrated in specific columns related to gaze, pupil size, distance, and camera position, suggesting they represent missing or invalid sensor readings.
- The `1` values, after mapping from `4.0` in the original validity columns, indicate instances of invalid eye-tracking data.
- The heatmaps also show a strong correlation between the `-1` values in the pupil columns and a validity of 1 in the newly created pupil validity features, confirming that -1 was used to mark invalid pupil data.

In [ ]:
df_38_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_38_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_38_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms

The grid of histograms provides insights into the distribution of values for each numeric column in the dataset (excluding 'UnixTime'). Key observations include:

- Several columns, such as `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, and `ET_GazeRighty`, show distributions that appear somewhat multimodal or skewed, suggesting variations in gaze patterns.
- The `ET_PupilLeft` and `ET_PupilRight` histograms clearly show a peak at -1, confirming the presence of a significant number of invalid pupil readings.
- `ET_TimeSignal` shows a relatively uniform distribution, as expected for a time-based signal.
- `ET_DistanceLeft` and `ET_DistanceRight` appear to have distributions centered around certain values, with some outliers or variations.
- The camera position columns (`ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`) seem to have distributions concentrated within specific ranges, reflecting the camera's field of view.
- The validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show distributions dominated by 0, indicating that most of the data is considered valid after the mapping. The smaller peaks at 1 represent the instances of invalid data.

These distributions highlight the need for appropriate handling of the -1 values and potential outliers in subsequent analysis or modeling steps.

In [ ]:
df_38_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_38_EYE['Timestamp'], df_38_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots

The line plots showing various features against the `Timestamp` reveal the temporal patterns and fluctuations in the eye-tracking data. Key observations include:

- **Gaze Coordinates (`ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`):** These plots show the changes in gaze position over time. We can observe periods of relatively stable gaze interspersed with rapid movements (saccades) and blinks or other events where the gaze data might be invalid (-1 values appear as gaps or spikes if not handled).
- **Pupil Size (`ET_PupilLeft`, `ET_PupilRight`):** The pupil size plots show variations over time. The presence of many -1 values is evident as flat lines at the bottom of the plot, indicating periods where pupil data was not recorded or was invalid.
- **Time Signal (`ET_TimeSignal`):** This plot shows a steady, increasing trend, as expected for a time-based signal.
- **Distance and Camera Position (`ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, `ET_CameraRightY`):** These plots show how the distance from the eye tracker and the camera positions change over time. Variations in these features can be related to head movements or changes in the user's position relative to the eye tracker.
- **Validity (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`):** These plots clearly show periods of invalid data (represented by 1) as spikes or plateaus, corresponding to instances where the eye tracker lost track of the eyes or the pupil data was marked as invalid.

Analyzing these time series plots is crucial for understanding the dynamics of the eye-tracking data and identifying patterns or anomalies that may require further investigation or specific handling during subsequent analysis.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_38_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_38_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

# Observations from Boxplots and Handling -1 Values

The boxplots provide a visual summary of the distribution and potential outliers for each numeric column. Key observations from the boxplots include:

- The boxplots for columns like `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY` clearly show the presence of -1 values as significant outliers, confirming our earlier observations from the heatmaps and histograms.
- The boxplots for the validity columns (`ET_ValidityLeft`, `ET_ValidityRight`, `ET_PupilLeft_validity`, `ET_PupilRight_validity`) show the discrete nature of these features, with the majority of data points at 0 (valid) and a smaller number at 1 (invalid).

Given the significant presence of -1 values, which represent invalid or missing data, especially in the pupil-related columns, we have decided to replace these -1 values with NaN to properly represent them as missing data. Subsequently, we will impute these missing values using the mean of each respective column. This approach helps to retain the data structure and allows for further analysis or modeling without the distortion caused by the -1 placeholders.

In [ ]:
df_38_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
df_38_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].mean()

In [ ]:
df_38_EYE[['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']].median()

In [ ]:
numeric_cols = df_38_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_38_EYE[col] = df_38_EYE[col].fillna(df_38_EYE[col].mean())

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_38_EYE.isnull(), cmap='viridis')
plt.title('Heatmap of Missing Values After Imputation')

plt.subplot(1, 2, 2)
sns.heatmap(df_38_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

# Handling Missing Values (Imputation)

As decided, we have replaced all the `-1` values with `NaN` to treat them as missing data. Subsequently, we have imputed these `NaN` values with the mean of their respective columns. The heatmap above, which was generated after the imputation, now shows no visible signs of `NaN` values, indicating that the imputation was successful.

In [ ]:
df_38_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_38_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_38_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Observations from Histograms After Imputation

The histograms generated after replacing the -1 values with the mean of each column show the distributions of the numeric features with the missing data handled. Key observations from these updated histograms include:

- The distinct peaks at -1, which were prominent in the histograms for several columns (e.g., pupil size, gaze coordinates, distance, and camera position) before imputation, are now replaced by a peak at the mean of each respective column.
- The distributions in many columns now appear more unimodal or show shifted modes compared to the original histograms.
- The histograms for the validity columns still show their bimodal distributions with peaks at 0 and 1, as these were handled separately.

These histograms provide an updated view of the data's distribution after handling the missing values, highlighting the impact of the imputation method on the data's characteristics.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_PupilLeft', 'ET_PupilRight',
       'ET_TimeSignal', 'ET_DistanceLeft', 'ET_DistanceRight',
       'ET_CameraLeftX', 'ET_CameraLeftY', 'ET_CameraRightX',
       'ET_CameraRightY', 'ET_ValidityLeft', 'ET_ValidityRight',
       'ET_PupilLeft_validity', 'ET_PupilRight_validity']

In [ ]:
for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_38_EYE['Timestamp'], df_38_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

# Observations from Time Series Plots After Imputation

The line plots generated after imputing the missing values with the mean show the temporal patterns of the features with the missing data handled. Key observations from these updated plots include:

- The gaps or flat lines at -1, which were prominent in the plots for columns like gaze coordinates, pupil size, distance, and camera position, are now filled by lines at the mean value of the respective columns.
- The plots for the validity columns remain the same as they were handled separately.
- The `ET_TimeSignal` plot still shows a steady increasing trend, as expected.

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(df_38_EYE.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

# Observations from Correlation Heatmap

The correlation heatmap provides a visual representation of the pairwise correlations between the numeric columns in the dataset. Key observations from the heatmap include:

- **High Positive Correlations:** We observe strong positive correlations (values close to 1) between:
  - `ET_GazeLeftx` and `ET_GazeRightx`: This is expected as the gaze positions of both eyes should be highly correlated when fixating on a point.
  - `ET_GazeLefty` and `ET_GazeRighty`: Similar to the x-coordinates, the y-coordinates of gaze should also be highly correlated.
  - `ET_PupilLeft` and `ET_PupilRight`: Pupil sizes of both eyes tend to change together in response to light and cognitive load.
  - `ET_DistanceLeft` and `ET_DistanceRight`: The distance from the eye tracker to each eye should be highly correlated.
  - `ET_CameraLeftX` and `ET_CameraRightX`, `ET_CameraLeftY` and `ET_CameraRightY`: The camera positions for both eyes are also expected to be highly correlated.
  - `UnixTime` and `ET_TimeSignal`: As previously noted, these two columns are almost perfectly linearly correlated, indicating redundancy.
  - `ET_ValidityLeft` and `ET_PupilLeft_validity`: There is a positive correlation, suggesting that when the overall left eye data is invalid, the left pupil data is also likely to be invalid.
  - `ET_ValidityRight` and `ET_PupilRight_validity`: Similar to the left eye, there is a positive correlation between the overall right eye validity and the right pupil validity.
- **Other Correlations:** We can also observe other varying degrees of correlations between different features, which can provide insights into the relationships between gaze behavior, pupil size, distance, and camera position. For example, there might be correlations between gaze coordinates and camera positions, reflecting head movements.
- **Low or Near-Zero Correlations:** Columns with low or near-zero correlations are relatively independent of each other.

Understanding these correlations is important for feature selection and for building models, as highly correlated features might indicate multicollinearity, while correlations between features can reveal underlying patterns in the data.

# Analysis of ET_TimeSignal and Decision to Drop

As observed in the time series plot and confirmed by the correlation heatmap, the `ET_TimeSignal` column exhibits a near-perfect linear relationship with both the `Timestamp` and `UnixTime` columns. This strong correlation (close to 1) suggests that `ET_TimeSignal` is essentially redundant and likely represents another form of time recording or a signal directly derived from the timestamp.

Including highly correlated features like this in a dataset can lead to issues such as multicollinearity in some statistical models, which can make it difficult to interpret the individual impact of each feature. Since the `Timestamp` column already provides the necessary temporal information, retaining `ET_TimeSignal` does not appear to add significant value for further analysis or modeling in most cases.

Therefore, based on its high correlation and lack of unique insight, we will proceed to drop the `ET_TimeSignal` column to simplify the dataset and potentially improve the performance and interpretability of future analyses.

In [ ]:
df_38_EYE.drop('ET_TimeSignal', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16, 10))
sns.pairplot(df_38_EYE)
plt.show()